In [1]:
import itertools
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

## Data

In [2]:
# https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset
url = 'https://raw.githubusercontent.com/dliviya/MATH441/main/diabeteas_clean.csv'
full_data = pd.read_csv(url)
full_data.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,0.0,1.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,3.0,5.0,30.0,0.0,1.0,4.0,6.0,8.0
1,0.0,1.0,1.0,1.0,26.0,1.0,1.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,12.0,6.0,8.0
2,0.0,0.0,0.0,1.0,26.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,10.0,0.0,1.0,13.0,6.0,8.0
3,0.0,1.0,1.0,1.0,28.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,3.0,0.0,3.0,0.0,1.0,11.0,6.0,8.0
4,0.0,0.0,0.0,1.0,29.0,1.0,0.0,0.0,1.0,1.0,...,1.0,0.0,2.0,0.0,0.0,0.0,0.0,8.0,5.0,8.0


In [3]:
# selecting a subset of column to run models on
cols = ['Diabetes_binary', 'HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker']
subset_data = full_data[cols].copy()

In [4]:
# Splitting data into features and response variables
X = subset_data.drop(columns=['Diabetes_binary'])
y = subset_data['Diabetes_binary']

# Training and Test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=2001)

In [5]:
# Preprocess numerical values (they are all numerical)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [6]:
X_train_scaled.shape[1]

5

## L0 Regularization

In [7]:
# get combinations of features
combinations = []
for r in range(X_train_scaled.shape[1]):
    combinations.extend(itertools.combinations(range(X_train_scaled.shape[1]), r+1))


In [8]:
# iterate over all combinations and fit logistic regression models
best_model = None
chosen_variables = None
most_nonzero = float('inf')
for c in combinations:
    # get features for the given combination
    X_train_subset = X_train_scaled[:, list(c)]
    X_test_subset = X_test_scaled[:, list(c)]
    
    # fit logistic regression model
    model = LogisticRegression(penalty=None)
    model.fit(X_train_subset, y_train)
    
    # count nonzero coefficients
    nonzero_count = np.count_nonzero(model.coef_)
    
    # update best model
    if nonzero_count < most_nonzero:
        best_model = model
        chosen_variables = c
        most_nonzero = nonzero_count     

In [9]:
best_model.coef_

array([[0.81034239]])

In [10]:
# select best features
best_features_ind = list(chosen_variables)
best_features = X.columns[best_features_ind]

In [11]:
y_pred_l0 = best_model.predict(X_test_scaled[:, best_features_ind])

In [12]:
# accuracy for all combinations
best_model = None
chosen_variables = None
most_nonzero = float('inf')
for c in combinations:
    # get features for the given combination
    X_train_subset = X_train_scaled[:, list(c)]
    X_test_subset = X_test_scaled[:, list(c)]
    
    # fit logistic regression model
    model = LogisticRegression(penalty=None)
    model.fit(X_train_subset, y_train)
    
    # count nonzero coefficients
    nonzero_count = np.count_nonzero(model.coef_)
    
    best_features_ind_all = list(list(c))
    best_features_all = X.columns[best_features_ind_all]
    print(best_features_all)
    y_pred_l0_all = model.predict(X_test_scaled[:, best_features_ind_all])
    accuracy_l0 = accuracy_score(y_test, y_pred_l0_all)
    print(accuracy_l0)   

Index(['HighBP'], dtype='object')
0.6879479441720106
Index(['HighChol'], dtype='object')
0.6428706148623161
Index(['CholCheck'], dtype='object')
0.520746887966805
Index(['BMI'], dtype='object')
0.6301395699735949
Index(['Smoker'], dtype='object')
0.5443228970199925
Index(['HighBP', 'HighChol'], dtype='object')
0.6879479441720106
Index(['HighBP', 'CholCheck'], dtype='object')
0.6896454168238401
Index(['HighBP', 'BMI'], dtype='object')
0.6914371935118823
Index(['HighBP', 'Smoker'], dtype='object')
0.6879479441720106
Index(['HighChol', 'CholCheck'], dtype='object')
0.6457940399849114
Index(['HighChol', 'BMI'], dtype='object')
0.6667766880422482
Index(['HighChol', 'Smoker'], dtype='object')
0.6428706148623161
Index(['CholCheck', 'BMI'], dtype='object')
0.6339588834402112
Index(['CholCheck', 'Smoker'], dtype='object')
0.5513956997359487
Index(['BMI', 'Smoker'], dtype='object')
0.6381082610335722
Index(['HighBP', 'HighChol', 'CholCheck'], dtype='object')
0.6896454168238401
Index(['HighBP', '

## L1 Regularization

In [13]:
l1_model = LogisticRegression(penalty='l1', solver='liblinear') # liblinear was used because the dataset is fairly small
l1_model.fit(X_train_scaled, y_train)

l1_selector = SelectFromModel(l1_model, prefit=True)
X_train_l1 = l1_selector.transform(X_train_scaled)
X_test_l1 = l1_selector.transform(X_test_scaled)

In [14]:
# model is trained again. helps identify the important features. 
# training will train the model with only the feature we want
model_l1 = LogisticRegression()
model_l1.fit(X_train_l1, y_train)

LogisticRegression()

In [15]:
y_pred_l1 = model_l1.predict(X_test_l1)

In [16]:
best_features_l1 = l1_selector.get_support()
best_feature_names_l1 = X.columns[best_features_l1]

## L2 Regularization

In [17]:
l2_model = LogisticRegression(penalty='l2', solver='liblinear') # liblinear was used because the dataset is fairly small
l2_model.fit(X_train_scaled, y_train)

l2_selector = SelectFromModel(l2_model, prefit=True)
X_train_l2 = l2_selector.transform(X_train_scaled)
X_test_l2 = l2_selector.transform(X_test_scaled)

In [18]:
model_l2 = LogisticRegression()
model_l2.fit(X_train_l2, y_train)

LogisticRegression()

In [19]:
y_pred_l2 = model_l2.predict(X_test_l2)

In [20]:
best_feature_l2 = l2_selector.get_support()
best_feature_names_l2 = X.columns[best_feature_l2]

### Accuracy Check

In [21]:
accuracy_l0 = accuracy_score(y_test, y_pred_l0)
accuracy_l1 = accuracy_score(y_test, y_pred_l1)
accuracy_l2 = accuracy_score(y_test, y_pred_l2)
print('L0: ', accuracy_l0)
print('Best Features for L0:', best_features.tolist())
print('L1: ', accuracy_l1)
print('Best Features for L1:', best_feature_names_l1.tolist())
print('L2: ', accuracy_l2)
print('Best Features for L2:', best_feature_names_l2.tolist())

L0:  0.6879479441720106
Best Features for L0: ['HighBP']
L1:  0.7036967182195398
Best Features for L1: ['HighBP', 'HighChol', 'CholCheck', 'BMI', 'Smoker']
L2:  0.702093549603923
Best Features for L2: ['HighBP', 'HighChol', 'BMI']
